In this note book, I have investigated the performance of three calssifiers (XGB, Stacked LSTM, and MLP) with class weight technique due to the exsiting imbalanced classe. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import f1_score, recall_score


In [ ]:
data= pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
print(data.head(5))
data.isnull().sum()


In [ ]:
data['bmi']=data['bmi'].fillna(data['bmi'].mean())

imbalancity check:

In [ ]:
print(len(data[data['stroke']==1]), len(data[data['stroke']==0]))

In [ ]:
np.unique(data['work_type'])

In [ ]:
np.unique(data['smoking_status'])

In [ ]:
data.gender.unique()

In [ ]:

categories=['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status']



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
for i in categories:
    cat=data[data['stroke']==1][i]
    fig,n=plt.subplots(figsize=(10,3))
    g=sns.barplot(x=cat.value_counts().index,y=cat.value_counts())
    g.set_title(f'Stroke distribution by {i}')
    g.set_ylabel('Number of observations')
   

In [ ]:
obtype=['gender','ever_married','work_type','Residence_type','smoking_status']
gen_encode=LabelEncoder()
mar_encode=LabelEncoder()
work_encode=LabelEncoder()
res_encode=LabelEncoder()
smo_encode=LabelEncoder()
data['gender']=gen_encode.fit_transform(data['gender'])
data['ever_married']=mar_encode.fit_transform(data['ever_married'])
data['work_type']=work_encode.fit_transform(data['work_type'])
data['Residence_type']=res_encode.fit_transform(data['Residence_type'])
data['smoking_status']=smo_encode.fit_transform(data['smoking_status'])

In [ ]:

X=data.iloc[:,1:-1]
y=data.iloc[:,-1]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4, random_state=42)
n_features= X.shape[1]

In [ ]:
data.head(3)

XGB Classifier:

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, recall_score

model = xgb.XGBClassifier(n_estimators= 50,objective= 'binary:logistic', scale_pos_weight=1500, seed=42)
model.fit(X_train, y_train )

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
print(f1_score(y_test, y_pred))

LSTM classifier:

In [ ]:
#LSTM model 
import os
import numpy as np
import pandas as pd
import random, math
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle as skshuffle
from sklearn.model_selection import train_test_split as sk_train_test_split
from tensorflow.keras.layers import Convolution1D, Dense, Flatten, GlobalAveragePooling1D, AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.layers import Dropout, AveragePooling2D, LSTM, GRU
from sklearn.metrics import mean_squared_error
from time import time
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

from tensorflow.keras import regularizers

os.environ["PYTHONHASHSEED"] = "0"
np.random.seed(64)
random.seed(64)
tf.compat.v1.set_random_seed(64)
from keras import backend as K
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
from tensorflow.compat.v1.keras import backend as K
K.set_session(sess)

In [ ]:
data.head(3)

In [ ]:
X_train= np.array(X_train).reshape(len(X_train), 1, n_features)
X_test= np.array(X_test).reshape(len(X_test), 1, n_features)


In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

In [ ]:
model2 = Sequential()
model2.add(LSTM(50, return_sequences=True, input_shape=(1, X_train.shape[2])))
# model.add(Dropout(0.2))
model2.add(LSTM(25, return_sequences=False))

model2.add(Dense(1, activation='sigmoid'))
from tensorflow.keras.optimizers import SGD
# model.compile(loss='binary_crossentropy', optimizer=SGD(), metrics=['binary_accuracy'])
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()
# fit model
model2.fit(X_train, y_train, epochs=20, batch_size=32, class_weight=class_weights, shuffle=True)
# 

In [ ]:
predictions= model2.predict_classes(X_test)


In [ ]:
sum(predictions)

In [ ]:
print(f1_score(y_test, predictions, average = 'binary'))


MLP model:

Reshaping the data properly:

In [ ]:
X_train = X_train.reshape(len(X_train), n_features)
X_test= X_test.reshape(len(X_test), n_features)

In [ ]:
#### MLP performance investigation

input_shape = (n_features,)
model3= Sequential()
model3.add(Dense(350, input_shape=input_shape, activation='relu'))
model3.add(Dense(50, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

# Configure the model and start training
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(X_train, y_train, epochs=20, batch_size=32,class_weight=class_weights, shuffle=True )

In [ ]:
prediction= model3.predict_classes(X_test)


In [ ]:
print(f1_score(y_test, prediction, average = 'binary'))
